In [ ]:
# ---------------------------------------------------------------
# Install required packages
# ---------------------------------------------------------------

!pip install pathway bokeh --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ---------------------------------------------------------------
# Import standard Python libraries and Pathway
# ---------------------------------------------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

import pathway as pw
import bokeh.plotting
import panel as pn


In [ ]:
# ---------------------------------------------------------------
# Load CSV and prepare for streaming into Pathway
# ---------------------------------------------------------------

# Read the original dataset
parking_df = pd.read_csv('dataset.csv')

# Create a single timestamp column by merging date and time
parking_df['Timestamp'] = pd.to_datetime(
    parking_df['LastUpdatedDate'] + ' ' + parking_df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort the dataframe chronologically
parking_df = parking_df.sort_values('Timestamp').reset_index(drop=True)

# Save a lightweight version of the CSV for streaming into Pathway
parking_df[[
    "Timestamp",
    "Occupancy",
    "Capacity",
    "QueueLength",
    "TrafficConditionNearby",
    "IsSpecialDay",
    "VehicleType"
]].to_csv("parking_stream.csv", index=False)


In [ ]:
# ---------------------------------------------------------------
# Define schema for reading data into Pathway
# ---------------------------------------------------------------

class ParkingLotSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str


In [ ]:
# ---------------------------------------------------------------
# Load the CSV into a Pathway streaming table
# ---------------------------------------------------------------

# Replay the CSV data as a streaming source
parking_table = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingLotSchema,
    input_rate=1000
)

# Format to parse timestamp strings into datetime objects
datetime_fmt = "%Y-%m-%d %H:%M:%S"

# Add a parsed datetime column
parking_table_with_time = parking_table.with_columns(
    timestamp_dt = parking_table.Timestamp.dt.strptime(datetime_fmt)
)


In [ ]:
# ---------------------------------------------------------------
# Define model hyperparameters and pricing bounds
# ---------------------------------------------------------------

base_price_value = 10.0

# Coefficients for demand calculation
weight_occupancy = 1.0
weight_queue = 0.1
weight_traffic = 0.5
weight_special_day = 0.3
weight_vehicle_type = 1.0

# Price sensitivity coefficient
price_lambda = 0.5

# Price floor and ceiling
min_price_allowed = 5.0
max_price_allowed = 20.0


In [ ]:
# ---------------------------------------------------------------
# Define User-Defined Functions for custom logic
# ---------------------------------------------------------------

@pw.udf
def compute_vehicle_weight(vehicle_type) -> float:
    """Assign a weight based on vehicle type."""
    return {
        'car': 0.3,
        'bike': 0.1,
        'truck': 0.5
    }.get(str(vehicle_type).lower(), 0.2)

@pw.udf
def compute_traffic_score(traffic_level) -> float:
    """Assign a numeric score to traffic levels."""
    return {
        'low': 0.1,
        'average': 0.5,
        'high': 1.0
    }.get(str(traffic_level).lower(), 0.5)

@pw.udf
def multiply_values(value, weight) -> float:
    """Multiply a value by a weight, handling None safely."""
    if value is None:
        return 0.0
    return float(value * weight)


In [ ]:
# ---------------------------------------------------------------
# Calculate intermediate demand terms for pricing model
# ---------------------------------------------------------------

# Compute occupancy ratio and factor scores
parking_table_with_demand = parking_table_with_time.with_columns(
    occupancy_ratio = pw.this.Occupancy / pw.this.Capacity,
    vehicle_type_score = compute_vehicle_weight(pw.this.VehicleType),
    traffic_condition_score = compute_traffic_score(pw.this.TrafficConditionNearby)
)

# Compute weighted terms for each component
parking_table_with_demand = parking_table_with_demand.with_columns(
    term_occupancy = multiply_values(parking_table_with_demand.occupancy_ratio, weight_occupancy),
    term_queue = multiply_values(pw.this.QueueLength, weight_queue),
    term_traffic = multiply_values(parking_table_with_demand.traffic_condition_score, weight_traffic),
    term_special_day = multiply_values(pw.this.IsSpecialDay, weight_special_day),
    term_vehicle_type = multiply_values(parking_table_with_demand.vehicle_type_score, weight_vehicle_type)
)


In [ ]:
# ---------------------------------------------------------------
# Compute raw demand value from weighted terms
# ---------------------------------------------------------------

# Raw demand is a linear combination of terms
parking_table_with_demand = parking_table_with_demand.with_columns(
    raw_demand = (
        parking_table_with_demand.term_occupancy
        + parking_table_with_demand.term_queue
        - parking_table_with_demand.term_traffic
        + parking_table_with_demand.term_special_day
        + parking_table_with_demand.term_vehicle_type
    )
)


In [ ]:
# ---------------------------------------------------------------
# Normalize demand into a bounded [0, 1] range
# ---------------------------------------------------------------

@pw.udf
def normalize_demand_value(demand_value) -> float:
    """Normalize demand so it stays between 0 and 1."""
    return min(1.0, max(0.0, demand_value / 5.0))  # assuming max raw demand ~5.0

parking_table_with_demand = parking_table_with_demand.with_columns(
    normalized_demand = normalize_demand_value(pw.this.raw_demand)
)


In [ ]:
# ---------------------------------------------------------------
# Calculate final dynamic price based on normalized demand
# ---------------------------------------------------------------

@pw.udf
def compute_dynamic_price(norm_demand) -> float:
    """Calculate price based on normalized demand."""
    new_price = base_price_value * (1 + price_lambda * norm_demand)
    # Enforce price bounds
    return min(max_price_allowed, max(min_price_allowed, new_price))

# Apply price calculation
parking_table_with_price = parking_table_with_demand.with_columns(
    dynamic_price = compute_dynamic_price(pw.this.normalized_demand)
)


In [ ]:
# ---------------------------------------------------------------
# Visualize the dynamic pricing time series with Bokeh
# ---------------------------------------------------------------

pn.extension()

def plot_dynamic_pricing(source_table):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Parking Pricing - Model 2",
        x_axis_type="datetime"
    )
    fig.line("timestamp_dt", "dynamic_price", source=source_table, line_width=2, color="navy")
    fig.scatter("timestamp_dt", "dynamic_price", source=source_table, size=6, color="red")
    return fig

# Create interactive visualization
viz_panel = parking_table_with_price.plot(plot_dynamic_pricing, sorting_col="timestamp_dt")
pn.Column(viz_panel).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# ---------------------------------------------------------------
# Execute the Pathway pipeline
# ---------------------------------------------------------------

%%capture --no-display
pw.run()


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished          6975        18306    output        18278       None                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    